In [148]:
from bs4 import BeautifulSoup
import pandas as pd

## Bài 2

In [149]:
# Đọc file NetworkConf
with open('../Lab3/data/NetworkConf.xml','r', encoding = 'utf-8') as f:
    NetworkConf = f.read()

# Đọc file TermConf
with open('../Lab3/data/TermConf.xml','r',encoding='utf-8') as f:
    TermConf = f.read()

In [150]:
# Lấy dữ liệu trong các tag cần thiết
NetworkConf = BeautifulSoup(NetworkConf,'xml')
NetworkConf = NetworkConf.find_all('NetworkSession')

TermConf = BeautifulSoup(TermConf,'xml')
TermConf = TermConf.find('Terminal')

### 2.1

#### Trích xuất dữ liệu

In [151]:
def get_data_from_NetworkConf(soup: BeautifulSoup) -> dict:

    '''Lấy các dữ liệu theo yêu cầu của đề bài trong file NetworkConf.xml'''
    groupProtpcol = {}

    for _ in soup:
        name = _.get('name').strip()
        protocol = _.get('protocol').strip()

        # Thuộc tính `hostId` nằm trong tag khác nên ta sử dụng `find` để truy cập vào tag đó
        NetworkAddress = _.find('NetworkAddress')

        # trước ':' là ip, sau ':' là port. vd: 10.30.1.65 : 8081
        Ip, Port = NetworkAddress.get('hostIP').strip().split(':')

        # `enable` chỉ có 2 giá trị
        dictEnable = {"0": "không kết nối",
                    "1": "đang kết nối"}
        enable = _.get('enable').strip()
        status = dictEnable.get(enable).strip()

        if protocol not in groupProtpcol:
            # Protocol sẽ làm key
            groupProtpcol[protocol] = []
        groupProtpcol[protocol].append((name, Ip, Port, enable, status))

    return groupProtpcol

In [152]:
def get_data_from_TermConf(soup:BeautifulSoup) -> list:

    '''Lấy các dữ liệu theo yêu cầu của đề bài trong file TermConf.xml'''
    lst_termConf = []

    id = soup.get('id').strip()
    
    # Các dữ liệu nằm trong tag con thì cần phải sử dụng find để truy cập
    TermIP = soup.find('TermIP').text.strip()
    branchNumber = soup.find('branchNumber').text.strip()
    TerminalType = soup.find('TerminalType').text.strip()

    lst_termConf.append([id, TermIP, branchNumber,TerminalType]) 

    return lst_termConf


#### Hiển thị dữ liệu

In [153]:
# Hiện thị dữ liệu theo yêu cầu trong file NetworkConf.xml
print(f"Hiện thị thông tin theo từng Protocol")
for _ in get_data_from_NetworkConf(NetworkConf):
    print(f"\nProtocol: {_}")

    for name, Ip, Port, enable, status in get_data_from_NetworkConf(NetworkConf)[_]:
        print(f"\tname: {name}, IP: {Ip}, Port: {Port}, enable:{enable}, status: {status}")
        
print('---'*25)
print(f"Thông tin của ATM\n")
for id, TermIP, branchNumber, TerminalType in get_data_from_TermConf(TermConf):
    print(f"ID: {id}\nIP: {TermIP}\nBranch Number: {branchNumber}\nTerminal Type: {TerminalType}")

Hiện thị thông tin theo từng Protocol

Protocol: ISO8583
	name: ATMP1, IP: 10.1.8.11, Port: 1234, enable:0, status: không kết nối
	name: ATMP3, IP: 21.96.3.137, Port: 9112, enable:0, status: không kết nối

Protocol: NDC/DDC
	name: ATMP2, IP: 10.30.1.65, Port: 8081, enable:1, status: đang kết nối

Protocol: xml/json
	name: RESTFUL1, IP: 10.1.24.86, Port: 8089, enable:0, status: không kết nối
	name: RESTFUL2, IP: 10.1.25.88, Port: 8088, enable:1, status: đang kết nối
---------------------------------------------------------------------------
Thông tin của ATM

ID: 00000011
IP: 10.1.12.15
Branch Number: 1234
Terminal Type: Auto Banking


---

### 2.2